In [1]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
from scipy import ndimage
import numpy as np
# define the dictionary of digit segments so we can identify
# each digit on the thermostat
DIGITS_LOOKUP = {
	(1, 1, 1, 0, 1, 1, 1): '0',
	(0, 0, 1, 0, 0, 1, 0): '1',
    (0, 0, 1, 0, 0, 1, 1): '1',  # it is 1 and point
	(1, 0, 1, 1, 1, 0, 1): '2',
	(1, 0, 1, 1, 0, 1, 1): '3',
	(0, 1, 1, 1, 0, 1, 0): '4',
    (0, 1, 1, 1, 0, 1, 1): '4', # it is 4 and point
	(1, 1, 0, 1, 0, 1, 1): '5',
	(1, 1, 0, 1, 1, 1, 1): '6',
	(1, 0, 1, 0, 0, 1, 0): '7',
    (1, 0, 1, 0, 0, 1, 1): '7', # it is 7 and point
	(1, 1, 1, 1, 1, 1, 1): '8',
	(1, 1, 1, 1, 0, 1, 1): '9'
}

C:\Users\andr\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def show(img):
    cv2.imshow('image', img)
    cv2.waitKey(0)

# load the example image


In [15]:
def digits_from_image(image):
    # pre-process the image by resizing it, converting it to
    # graycale, blurring it, and computing an edge map
    image = imutils.resize(image, height=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(blurred, 50, 170, 255)
    # show(edged)
    # find contours in the edge map, then sort them by their
    # size in descending order
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    displayCnt = None
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if the contour has four vertices, then we have found
        # the thermostat display
        if len(approx) == 4:
            displayCnt = approx
            break
    if displayCnt is None:
        #show(edged)
        return '???'
    # extract the thermostat display, apply a perspective transform
    # to it
    warped = four_point_transform(gray, displayCnt.reshape(4, 2))
    output = four_point_transform(image, displayCnt.reshape(4, 2))
    # threshold the warped image, then apply a series of morphological
    # operations to cleanup the thresholded image
    thresh = cv2.inRange(output, (0, np.average(output[:, :, 1])*2, 0), (130, 255, 130))

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 3))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    
    thresh = cv2.GaussianBlur(thresh, (3, 3), 0)
    # show(thresh)
    # show(image)
    # find contours in the thresholded image, then initialize the
    # digit contours lists
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    digitCnts = []
    # loop over the digit area candidates
    for c in cnts:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)
        # if the contour is sufficiently large, it must be a digit
        # print(x,y,w,h)
        if w >= 5 and (h >= 10 and h <= 50):
            digitCnts.append(c)
    # sort the contours from left-to-right, then initialize the
    # actual digits themselves
    if len(digitCnts) == 0:
        return "???"
    digitCnts = contours.sort_contours(digitCnts,
        method="left-to-right")[0]
    digits = []
    # loop over each of the digits
    digits = []
    for c in digitCnts:
        # extract the digit ROI
        (x, y, w, h) = cv2.boundingRect(c)
        if h >= w*2.7:
            digits.append('1')
            continue

        roi = thresh[y:y + h, x:x + w]
        # show(ndimage.rotate(roi, 3))
        # compute the width and height of each of the 7 segments
        # we are going to examine
        (roiH, roiW) = roi.shape
        (dW, dH) = (int(roiW * 0.3), int(roiH * 0.2))
        dHC = int(roiH * 0.05)
        # define the set of 7 segments
        segments = [
            ((0, 0), (w, dH)),	# top
            ((0, 0), (dW, h // 2)),	# top-left
            ((w - dW, 0), (w, h // 2)),	# top-right
            ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # center
            ((0, h // 2), (dW, h)),	# bottom-left
            ((w - dW, h // 2), (w, h)),	# bottom-right
            ((0, h - dH), (w, h))	# bottom
        ]
        on = [0] * len(segments)
            # loop over the segments
        for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
            # extract the segment ROI, count the total number of
            # thresholded pixels in the segment, and then compute
            # the area of the segment
            segROI = ndimage.rotate(roi, 3)[yA:yB, xA:xB]
            total = cv2.countNonZero(segROI)
            area = (xB - xA) * (yB - yA)
            # if the total number of non-zero pixels is greater than
            # 50% of the area, mark the segment as "on"
            if float(area) > 0 and total / float(area) > 0.5:
                on[i]= 1
        # lookup the digit and draw it on the image
        if tuple(on) not in DIGITS_LOOKUP:
            digits.append('?')
            continue
        digit = DIGITS_LOOKUP[tuple(on)]
        digits.append(digit)
        #cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #cv2.putText(output, str(digit), (x - 10, y - 10),
        #	cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
    # show(output)
    return ''.join(digits)

In [4]:
digits_from_image(cv2.imread('test3.png'))

'4511'

In [53]:
digits_from_image(cv2.imread('test7.png'))

42 78
41 72
41 84


'4921'

In [21]:
import csv

def parse_video(name, f='.MOV'):
    cap = cv2.VideoCapture("videos/" + name + f, apiPreference=cv2.CAP_MSMF)
    fps = cap.get(cv2.CAP_PROP_FPS)
    n = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    dat = []
    c = 0

    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break

        dat.append((c/fps, digits_from_image(frame)))
        #print(dat[-1][1])
        if dat[-1][1] == '1000': # boost if nothing interesting
            cap.read()
            cap.read()
            c += 2
            if c >= 1000:
                print(c,'/', int(n))
                c -= 1000
        c += 1
        if not c%1000:
            print(c,'/', int(n))
    cap.release()

    with open(f"data/{name}.csv", "wt") as fp:
        writer = csv.writer(fp, delimiter=";")
        # writer.writerow(["your", "header", "foo"])  # write header
        writer.writerows(dat)

In [48]:
parse_video('IMG_5567')


1000 / 2302
2000 / 2302


In [49]:
parse_video('IMG_5568')

In [50]:
parse_video('IMG_5569')

1000 / 3376
2000 / 3376
3000 / 3376


In [51]:
parse_video('IMG_5570')

1000 / 4182
2000 / 4182
3000 / 4182
4000 / 4182


In [52]:
parse_video('IMG_5571')

1000 / 2829
2000 / 2829


In [13]:
parse_video('ZERO', f='.mp4')

1000 / 35963
2000 / 35963
3000 / 35963
4000 / 35963
5000 / 35963
6000 / 35963
7000 / 35963
8000 / 35963
9000 / 35963
10000 / 35963
11000 / 35963
12000 / 35963
13000 / 35963
14000 / 35963
15000 / 35963
16000 / 35963
17000 / 35963
18000 / 35963
19000 / 35963
20000 / 35963
21000 / 35963
22000 / 35963
23000 / 35963
24000 / 35963
25000 / 35963
26000 / 35963
27000 / 35963
28000 / 35963
29000 / 35963
30000 / 35963
31000 / 35963
32000 / 35963
33000 / 35963
34000 / 35963
35000 / 35963
35889 / 35963
35892 / 35963
35895 / 35963
35898 / 35963
35901 / 35963
35904 / 35963
35907 / 35963
35910 / 35963
35913 / 35963
35916 / 35963
35919 / 35963
35922 / 35963
35925 / 35963
35928 / 35963
35931 / 35963
35934 / 35963
35937 / 35963
35940 / 35963
35943 / 35963
35946 / 35963
35949 / 35963
35952 / 35963
35955 / 35963
35958 / 35963
35961 / 35963


In [16]:
parse_video('I', f='.mp4')

1000 / 14426
2000 / 14426
3000 / 14426
4000 / 14426
5000 / 14426
6000 / 14426
7000 / 14426
8000 / 14426
9000 / 14426
10000 / 14426
11000 / 14426
12000 / 14426
13000 / 14426
14000 / 14426
14264 / 14426
14267 / 14426
14270 / 14426
14273 / 14426
14276 / 14426
14279 / 14426
14282 / 14426
14285 / 14426
14288 / 14426
14291 / 14426
14294 / 14426
14297 / 14426
14300 / 14426
14303 / 14426
14306 / 14426
14309 / 14426
14312 / 14426
14315 / 14426
14318 / 14426
14321 / 14426
14324 / 14426
14327 / 14426
14330 / 14426
14333 / 14426
14336 / 14426
14339 / 14426
14342 / 14426
14345 / 14426
14348 / 14426
14351 / 14426
14355 / 14426
14358 / 14426
14362 / 14426
14365 / 14426
14368 / 14426
14371 / 14426
14374 / 14426
14377 / 14426
14380 / 14426
14383 / 14426
14387 / 14426
14390 / 14426
14393 / 14426
14396 / 14426
14399 / 14426
14402 / 14426
14406 / 14426
14409 / 14426
14412 / 14426
14415 / 14426
14418 / 14426
14421 / 14426
14424 / 14426
14427 / 14426


In [17]:
parse_video('II', f='.mp4')

1000 / 33147
2000 / 33147
3000 / 33147
4000 / 33147
5000 / 33147
6000 / 33147
7000 / 33147
8000 / 33147
9000 / 33147
10000 / 33147
11000 / 33147
12000 / 33147
13000 / 33147
14000 / 33147
15000 / 33147
16000 / 33147
17000 / 33147
18000 / 33147
19000 / 33147
20000 / 33147
21000 / 33147
22000 / 33147
23000 / 33147
24000 / 33147
25000 / 33147
26000 / 33147
27000 / 33147
28000 / 33147
29000 / 33147
30000 / 33147
31000 / 33147
32000 / 33147
33000 / 33147


In [19]:
parse_video('III_1', f='.mp4')

1000 / 1969
1710 / 1969
1713 / 1969
1716 / 1969
1719 / 1969
1722 / 1969
1725 / 1969
1728 / 1969
1731 / 1969
1734 / 1969
1737 / 1969
1740 / 1969
1743 / 1969
1746 / 1969
1749 / 1969
1752 / 1969
1755 / 1969
1758 / 1969
1761 / 1969
1764 / 1969
1767 / 1969
1770 / 1969
1773 / 1969
1776 / 1969
1779 / 1969
1782 / 1969
1785 / 1969
1788 / 1969
1791 / 1969
1794 / 1969
1797 / 1969
1800 / 1969
1803 / 1969
1806 / 1969
1809 / 1969
1812 / 1969
1815 / 1969
1818 / 1969
1821 / 1969
1824 / 1969
1827 / 1969
1830 / 1969
1833 / 1969
1836 / 1969
1839 / 1969
1842 / 1969
1845 / 1969
1848 / 1969
1851 / 1969
1854 / 1969
1857 / 1969
1860 / 1969
1863 / 1969
1866 / 1969
1869 / 1969
1872 / 1969
1875 / 1969
1878 / 1969
1881 / 1969
1884 / 1969
1887 / 1969
1890 / 1969
1893 / 1969
1896 / 1969
1899 / 1969
1902 / 1969
1905 / 1969
1908 / 1969
1911 / 1969
1914 / 1969
1917 / 1969
1920 / 1969
1923 / 1969
1926 / 1969
1929 / 1969
1932 / 1969
1935 / 1969
1938 / 1969
1941 / 1969
1944 / 1969
1947 / 1969
1950 / 1969
1953 / 1969
1956

In [20]:
parse_video('III_2', f='.mp4')

1000 / 18565
2000 / 18565
3000 / 18565
4000 / 18565
5000 / 18565
6000 / 18565
7000 / 18565
8000 / 18565
9000 / 18565
10000 / 18565
11000 / 18565
12000 / 18565
13000 / 18565
14000 / 18565
15000 / 18565
16000 / 18565
17000 / 18565
17909 / 18565
17912 / 18565
17915 / 18565
17918 / 18565
17921 / 18565
17924 / 18565
17927 / 18565
17930 / 18565
17933 / 18565
17936 / 18565
17939 / 18565
17942 / 18565
17945 / 18565
17948 / 18565
17951 / 18565
17954 / 18565
17957 / 18565
17960 / 18565
17963 / 18565
17966 / 18565
17969 / 18565
17972 / 18565
17975 / 18565
17978 / 18565
17981 / 18565
17984 / 18565
17987 / 18565
17990 / 18565
17993 / 18565
17996 / 18565
17999 / 18565
18000 / 18565
18002 / 18565
18005 / 18565
18008 / 18565
18011 / 18565
18014 / 18565
18017 / 18565
18020 / 18565
18023 / 18565
18026 / 18565
18029 / 18565
18032 / 18565
18035 / 18565
18038 / 18565
18041 / 18565
18044 / 18565
18047 / 18565
18050 / 18565
18053 / 18565
18056 / 18565
18059 / 18565
18062 / 18565
18065 / 18565
18068 / 18565
1